<a href="https://colab.research.google.com/github/dbamman/nlp23/blob/main/HW5/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 5: WordNet

In this homework, you will be working with WordNet synsets and exploring methods to align new words (not in WordNet) with an existing synset.

In [ ]:
!wget https://people.ischool.berkeley.edu/~dbamman/glove.6B.100d.100K.txt
!pip install sentence_transformers

import nltk
import math
from nltk import word_tokenize
from nltk.corpus import wordnet as wn
import numpy as np
from typing import Callable, List, Tuple, Dict
from IPython.display import Markdown, display
from sentence_transformers import SentenceTransformer

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

def test_print(string):
    display(Markdown(string))

--2023-03-22 03:04:18--  https://people.ischool.berkeley.edu/~dbamman/glove.6B.100d.100K.txt
Resolving people.ischool.berkeley.edu (people.ischool.berkeley.edu)... 128.32.78.16
Connecting to people.ischool.berkeley.edu (people.ischool.berkeley.edu)|128.32.78.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85951834 (82M) [text/plain]
Saving to: ‘glove.6B.100d.100K.txt’

glove.6B.100d.100K. 100%[===================>]  81.97M   112MB/s    in 0.7s    

2023-03-22 03:04:19 (112 MB/s) - ‘glove.6B.100d.100K.txt’ saved [85951834/85951834]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


## Preliminaries: WordNet in NLTK

NLTK provides a great interface to the WordNet ontology.  Remember that core unit within WordNet is the **synset** (a category of near-synonyms).  A word (like "blue") can appear in many different synsets, each corresponding to a distinct *sense* of that word.

In [ ]:
# get all of the synsets that a specific word belongs to; print their definitions
synsets=wn.synsets('blue')
for synset in synsets:
    print(synset, synset.definition())

Synset('blue.n.01') blue color or pigment; resembling the color of the clear sky in the daytime
Synset('blue.n.02') blue clothing
Synset('blue.n.03') any organization or party whose uniforms or badges are blue
Synset('blue_sky.n.01') the sky as viewed during daylight
Synset('bluing.n.01') used to whiten laundry or hair or give it a bluish tinge
Synset('amobarbital_sodium.n.01') the sodium salt of amobarbital that is used as a barbiturate; used as a sedative and a hypnotic
Synset('blue.n.07') any of numerous small butterflies of the family Lycaenidae
Synset('blue.v.01') turn blue
Synset('blue.s.01') of the color intermediate between green and violet; having a color similar to that of a clear unclouded sky; - Helen Hunt Jackson
Synset('blue.s.02') used to signify the Union forces in the American Civil War (who wore blue uniforms)
Synset('gloomy.s.02') filled with melancholy and despondency
Synset('blasphemous.s.02') characterized by profanity or cursing
Synset('blue.s.05') suggestive of 

In [ ]:
# get all of the synsets that a specific word belongs to; print their definitions
synsets=wn.synsets('earnest')
for synset in synsets:
    print(synset, synset.definition())

Synset('earnest.n.01') something of value given by one person to another to bind a contract
Synset('earnest.s.01') characterized by a firm and humorless belief in the validity of your opinions
Synset('dear.s.03') earnest
Synset('businesslike.s.02') not distracted by anything unrelated to the goal


Any given synset will likewise contain multiple different words (all near-synonyms of each other).

In [ ]:
# get all of the words/phrase in a given synset
for lemma in wn.synset("gloomy.s.02").lemmas():
    print(lemma.name())

gloomy
grim
blue
depressed
dispirited
down
downcast
downhearted
down_in_the_mouth
low
low-spirited


Remember also that one of the powerful things about WordNet is that it places synsets within a hierarchical structure; a given synset has both **hypernyms** (other synsets that it is a subclass of) and **hyponyms** (other synsets that are subclasses of it).

In [ ]:
# Functions from http://www.nltk.org/howto/wordnet.html to get *all* of a synset's hyponym/hypernyms

hypo=lambda s: s.hyponyms()
hyper=lambda s: s.hypernyms()

Find all of the synsets that are hyponyms of the target synset (descendents in the WordNet hierarchy)

In [ ]:
list(wn.synset("blue.n.01").closure(hypo))

[Synset('azure.n.01'),
 Synset('dark_blue.n.01'),
 Synset('greenish_blue.n.01'),
 Synset('powder_blue.n.01'),
 Synset('prussian_blue.n.02'),
 Synset('purplish_blue.n.01'),
 Synset('steel_blue.n.01'),
 Synset('ultramarine.n.02')]

Find all of the synsets that are hyperyms (ancestors up the tree) of the target synset

In [ ]:
list(wn.synset("blue.n.01").closure(hyper))

[Synset('chromatic_color.n.01'),
 Synset('color.n.01'),
 Synset('visual_property.n.01'),
 Synset('property.n.02'),
 Synset('attribute.n.02'),
 Synset('abstraction.n.06'),
 Synset('entity.n.01')]

Here's how you can access all of the synsets in WordNet through NLTK (though note executing this may take a while, so it's commented out).

In [ ]:
for idx, synset in enumerate(wn.all_synsets()):
    print(idx, synset)
    if (idx > 10): break

0 Synset('able.a.01')
1 Synset('unable.a.01')
2 Synset('abaxial.a.01')
3 Synset('adaxial.a.01')
4 Synset('acroscopic.a.01')
5 Synset('basiscopic.a.01')
6 Synset('abducent.a.01')
7 Synset('adducent.a.01')
8 Synset('nascent.a.01')
9 Synset('emergent.s.02')
10 Synset('dissilient.s.01')
11 Synset('parturient.s.02')


## Programming deliverables

### Motivation and setup

WordNet is a great resource, but one of its downsides is *coverage* -- many of the words in our vocabulary aren't in WordNet, but could conceivably be placed within existing synsets within it.  Your task for this homework is to develop two methods to finding the closest synset for a given new word from Urban Dictionary.

For the scope of this homework, we're only going to pretend that WordNet only has 12 different synsets within it, as stored in `target_synsets` below. (Though, feel free to use the `wn.all_synsets()` function above if you wanted to explore running it on all of WordNet).

In [ ]:
target_synsets=[
    'spread.n.01',
    'formidable.s.01',
    'coziness.n.01',
    'mutation.n.02',
    'kernel.n.03',
    'faineant.s.01',
    'fund-raise.v.01',
    'orientation.n.06',
    'inappropriate.a.01',
    'stranger.n.02',
    'plausibility.n.01',
    'sever.v.01'
]

The code below demonstrates how you can access the definitions in each synset:

In [ ]:
for synset in target_synsets:
    wn_synset=wn.synset(synset)
    print(wn_synset)
    print("\tDefinition:", wn_synset.definition())

Synset('spread.n.01')
	Definition: process or result of distributing or extending over a wide expanse of space
Synset('formidable.s.01')
	Definition: extremely impressive in strength or excellence
Synset('coziness.n.01')
	Definition: a state of warm snug comfort
Synset('mutation.n.02')
	Definition: (genetics) any event that changes genetic structure; any alteration in the inherited nucleic acid sequence of the genotype of an organism
Synset('kernel.n.03')
	Definition: the choicest or most essential or most vital part of some idea or experience
Synset('faineant.s.01')
	Definition: disinclined to work or exertion
Synset('fund-raise.v.01')
	Definition: raise money for a cause or project
Synset('orientation_course.n.01')
	Definition: a course introducing a new situation or environment
Synset('inappropriate.a.01')
	Definition: not suitable for a particular occasion etc
Synset('stranger.n.02')
	Definition: an individual that one is not acquainted with
Synset('plausibility.n.01')
	Definition:

Here are the words that do not exist in WordNet now but that we want to add.  Each element of the tuple is:
```
(word, definition)
```

In [ ]:
urban_dictionary_terms: List[Tuple[str, str]]=[
    ("Crowdfunding", "the practice of obtaining needed funding (as for a new business) by soliciting contributions from a large number of people especially from the online community"),
    ("Hygge", "a cozy quality that makes a person feel content and comfortable"),
    ("biohacking", "biological experimentation (as by gene editing or the use of drugs or implants) done to improve the qualities or capabilities of living organisms especially by individuals and groups working outside a traditional medical or scientific research environment"),
    ("TL;DR", "a briefly expressed main point or key message that summarizes a longer discussion or explanation"),
    ("Hellacious", "Exceptionally powerful or violent; remarkably good; extremely difficult; extraordinarily large"),
    ("Unfriend", "To remove from one's list of friends (e.g. on a social networking website)"),
    ("Infodemic", "A wide and rapid spread of misinformation through various media, namely social media"),
    ("Onboarding", "The act or process of orienting and training a new employee"),
    ("Truthiness", "something that seems true but isn’t backed up by evidence"),
    ("Amotivational", "Relating to, or characterised by, a lack of motivation"),
    ("NSFW", "Not Safe For Work. Used to describe Internet content generally inappropriate for the typical workplace, i.e., would not be acceptable in the presence of your boss and colleagues"),
    ("Rando", "a person who is not known or recognizable or whose appearance (as in a conversation or narrative) seems unprompted or unwelcome")
]

### Overview

Your task here is to develop two different methods for finding the best matching synset:

**Deliverabe 1**. Find the WordNet synset with the highest cosine similarity between a) the average GloVe embeddings of its synset definition and b) the average GloVe embeddings of the new word definition.

**Deliverable 2**. Find the WordNet synset with the highest cosine similarity between a) the *sentence embedding* of its synset definition and b) the sentence embedding of the new word definition.

And evaluate them:

**Deliverable 3**. Implement a function `accuracy` that assesses quality of the dictionaries you return from `method_one` and `method_two`.


<font color='red'><b>Please note:</b></font>

<font color='red'><b>Do not change function signatures or return variable names</b>; if you believe it's necessary, let us know.

<font color='red'><b>Put your solutions between `# BEGIN SOLUTION` and `# END SOLUTION`</b>.</font>

Here is some code for reading in GloVe embeddings:


In [ ]:
def read_glove_vectors(filename: str):
    vocab_map={}
    embeddings=[]
    with(open(filename, encoding="utf-8")) as file:
        for idx, line in enumerate(file):
            cols=line.rstrip().split(" ")
            word=cols[0]
            embedding=cols[1:]

            embeddings.append(embedding)
            vocab_map[word]=idx

    return vocab_map, np.array(embeddings, dtype="float")

In [ ]:
glove_vocab_map, glove_embeddings=read_glove_vectors("glove.6B.100d.100K.txt")

Here is some code for loading the sentence transformer package:

In [ ]:
sentence_model=SentenceTransformer('sentence-transformers/all-distilroberta-v1')

Let's make sure it's working properly; the cell below should output this:
```
(768,)
```

In [ ]:
sentence_vector=sentence_model.encode("this is a sentence")
print(sentence_vector.shape)

(768,)


Here's an implementation of cosine similarity that you will find useful.

In [ ]:
def cosine_similarity(one, two):
    return np.dot(one, two) / (np.linalg.norm(one) * np.linalg.norm(two))

### Deliverable 1 (TODO). Implement the first method as `method_one` below.

As mentioned above, you should:

- a) Compute the average GloVe embedding of the Urban Dictionary word definition, and
- b) Use cosine similarity to compare it with the average GloVe embedding of the synset definitions.
- c) For each UD word, choose the definition that maximizes the cosine similarity with its definition.

Your function should return a dictionary mapping each urban dictionary term to a WordNet synset ID, e.g.:

```{
 "adore" : "love.v.01",
 "dripping" : "stylish.a.01"    
 }
```

#### Q1 (a). calculate the average GloVe embedding

Here are some things you need to do when calculating the average GloVe embedding of a sentence:

- Use `nltk.word_tokenize()` to tokenize the sentence.
- Treat everything as lowercase.
- Skip any tokens which don't appear in the GloVe vocabulary.
- Calculate the average value of the embedding vectors, which will be another vector of the same shape.

Implement this in `get_average_embedding()` below.

In [ ]:
def sentence_to_average_glove(sentence):
    global glove_embeddings, glove_vocab_map
    # BEGIN SOLUTION
    embeddings = []
    words = nltk.word_tokenize(sentence)

    for word in words:
        word=word.lower()
        if word in glove_vocab_map:
            embedding = glove_embeddings[glove_vocab_map[word]]
            embeddings.append(embedding)

    average_embedding = np.average(embeddings, axis=0) # TODO
    # END SOLUTION
    return average_embedding

In [ ]:
def public_test_q1a(student_sol_func):
    test_str='test sentence here'
    q1a=np.array(
        [-0.13161   ,  0.30526   ,  0.48708833, -0.01811933, -0.281338  ,
        0.2051    , -0.09271333,  0.61335667, -0.86006667,  0.46980667,
       -0.3530206 ,  0.34592667, -0.11837333,  0.174864  ,  0.227699  ,
        0.022927  ,  0.082307  ,  0.05022667, -0.615689  ,  0.04992267,
        0.43998   , -0.32244   ,  0.17315   ,  0.07476863,  0.24882667,
        0.20515667,  0.04977933, -0.35719133,  0.046468  ,  0.2846    ,
       -0.14525333,  0.17086367, -0.20106   , -0.16981597,  0.24622667,
        0.307406  , -0.43141   , -0.0322    , -0.32304733, -0.22139667,
       -0.69949667, -0.10295667,  0.58206667, -0.67504667,  0.18417   ,
       -0.10986333,  0.59897667, -0.75551   , -0.25218   , -0.59849   ,
        0.60331   , -0.4762    ,  0.19722333,  1.26916667, -0.39379333,
       -1.79474333, -0.631325  , -0.11708333,  1.6112    ,  0.24849667,
       -0.45793   ,  0.36410333, -0.16572   , -0.32961333,  0.45810667,
        0.20734   ,  0.27776   ,  0.41938233, -0.49610333, -0.04512333,
        0.10130303,  0.09548   ,  0.04292333, -0.24321   ,  0.18565   ,
        0.56445   ,  0.05074167, -0.512398  , -0.69699   , -0.19803   ,
        0.51716667, -0.28587633, -0.37449667, -0.37457667, -1.43945667,
        0.01966433,  0.06207733, -0.20610567, -0.30307667, -0.03681667,
       -0.23645   , -0.01583333, -0.07095133,  0.26235   ,  0.2177    ,
        0.43546   ,  0.20054333, -0.28842667,  0.11428867,  0.16190333])

    np.testing.assert_almost_equal(student_sol_func(test_str), q1a)
    test_print('**q1a** passed sanity check!')

In [ ]:
public_test_q1a(sentence_to_average_glove)

**q1a** passed sanity check!

#### Q1 (b). compare similarities

Use cosine similarity to compare it with the average GloVe embedding of the synset definitions.

- The input should be a) a sentence encoder, here `sentence_to_average_glove()` you just implemented (in q2 we'll be reusing this function, and pass a different encoder function there), b) an Urban Dictionary term, c) its definition, and d) your target WordNet synsets.
- You will populate the dictionary `term_similarity_scores` provided; the key will be a WordNet synset and the value the similarity score.
- You will need to sort the similairty scores, which will result in a list of key–value tuples.
- Unless you pass `silent=True`, the entries with top 3 similarity scores will be printed for your reference.

In [ ]:
def get_term_similarity_scores(sentence_encoder, urban_dictionary_word, urban_dictionary_definition, target_synsets, silent=False):
    global glove_embeddings, glove_vocab_map
    term_similarity_scores={}
    # BEGIN SOLUTION
    urban_encode = sentence_encoder(urban_dictionary_definition)

    for synset in target_synsets:
        synset_definition = wn.synset(synset).definition()
        synset_embedding = sentence_encoder(synset_definition)
        similarity = cosine_similarity(urban_encode, synset_embedding)
        term_similarity_scores[synset] = similarity

    term_similarity_scores = sorted(term_similarity_scores.items(), key=lambda x: x[1], reverse=True) # TODO
    # END SOLUTION
    if not silent:
        for k, v in term_similarity_scores[:3]:
            print(f"UD word: {urban_dictionary_word} -- WordNet synset: {k}, similarity:{v}")
    return term_similarity_scores

In [ ]:
def public_test_q1b(student_sol_func):
    q1b_test_term=('language', 'Language: A term in which if you ever curse around Steve, you will hear regularly.')
    q1b_test_synsets=['language.n.01', 'natural.s.04']
    np.testing.assert_almost_equal(student_sol_func(sentence_to_average_glove,
                                                    q1b_test_term[0],
                                                    q1b_test_term[1],
                                                    q1b_test_synsets, silent=True)[-1][-1], 0.8387250755283658)

    test_print('**q1b** passed sanity check!')

#### Sanity check for q1b

In [ ]:
public_test_q1b(get_term_similarity_scores)

**q1b** passed sanity check!

#### Q1 (c). bringing it all together

- For each UD word, choose the definition that maximizes the cosine similarity with its definition.

- Your function should return a dictionary mapping each urban dictionary term to a WordNet synset ID, e.g.:

```{
 "adore" : "love.v.01",
 "dripping" : "stylish.a.01"    
 }
```

In [ ]:
def method_one(sentence_encoder,
               urban_dictionary_terms: List[Tuple[str, str]],
               target_synsets: List[str]):
    """
    Method 1: an algorithm based on GloVe embeddings that maps each urban dictionary term to a synset ID.

    Parameters
    ----------
    sentence_encoder :
        a function to encode a sentence into word embeddings
        For q1, you should pass the function `sentence_to_average_glove` implemented in q1a here
    urban_dictionary_terms : List[Tuple[str, str]]
        a list of string 2-tuples where the first elements are words, second elements are definitions.
    target_synsets : List[str]
        a list of synset IDs that the words should be classified into.
        You can call `wn.synset("<synset ID>")` to get the synset object.

    Returns
    --------
    A dictionary mapping each urban dictionary term to a WordNet synet ID, e.g.
    `{"adore" : "love.v.01", "dripping" : "stylish.a.01"}`

    """
    best_matches={}
    for word, definition in urban_dictionary_terms:
        # BEGIN SOLUTION
        cosine_sword = get_term_similarity_scores(sentence_encoder, word, definition, target_synsets, silent=True)
        best_matches[word] = cosine_sword[0][0] # TODO
        # END SOLUTION
    return best_matches


In [ ]:
method_one_results=method_one(sentence_to_average_glove, urban_dictionary_terms, target_synsets)
method_one_results

{'Crowdfunding': 'spread.n.01',
 'Hygge': 'stranger.n.02',
 'biohacking': 'kernel.n.03',
 'TL;DR': 'stranger.n.02',
 'Hellacious': 'formidable.s.01',
 'Unfriend': 'stranger.n.02',
 'Infodemic': 'spread.n.01',
 'Onboarding': 'orientation.n.06',
 'Truthiness': 'stranger.n.02',
 'Amotivational': 'kernel.n.03',
 'NSFW': 'stranger.n.02',
 'Rando': 'stranger.n.02'}

### Deliverable 2 (TODO). Implement your second method as `method_two` below.

In this function, you should compute the cosine similarity between the sentence embedding of the UD word definition and those of the synsets, then for each UD word, choose the synset with the highest cosine similarity.

For consistency, use the sentence transformer model called `sentence-transformers/all-distilroberta-v1` stored in `sentence_model` from earlier.

You *must* add all similarity scores to the list `all_term_similarity_scores` provided.

Your function must also return a dictionary mapping each urban dictionary term to a WordNet synset ID, e.g.:

```
{
 "adore" : "love.v.01",
 "dripping" : "stylish.a.01"    
 }
```

#### Q2 (a). use sentence transformer to encode sentences

In [ ]:
# make sure you have `sentence_model`
sentence_model=SentenceTransformer('sentence-transformers/all-distilroberta-v1')

Use `sentence_model` to obtain the sentence embedding of the input sentence. You can do this with one line of code.

In [ ]:
def sentence_to_sentence_embedding(sentence):
    global sentence_model
    # BEGIN SOLUTION
    return sentence_model.encode(sentence) # TODO
    # Comment this line out before you start:
    # return NotImplementedError
    # END SOLUTION

In [ ]:
def public_test_q2a(student_sol_func):
    test_str='test sentence here'
    q2a=np.array([-1.43969897e-02, -3.58152948e-02,  7.82628136e-04,  6.85324892e-03, 5.31034134e-02, -1.05041144e-02,  3.62984999e-03,  3.53333056e-02, 5.01355790e-02, -4.29266617e-02, -4.22659479e-02, -7.64900371e-02, 4.23901081e-02, -5.56682684e-02, -4.78991754e-02,  8.80383980e-03, 3.55795096e-03, -3.28393700e-03,  3.95287480e-03,  1.80463120e-02, 7.05414824e-03,  5.80327772e-02,  4.47773710e-02, -4.21631448e-02, 7.39850774e-02,  1.79811213e-02,  7.04950839e-02, -4.73053604e-02, -4.15716507e-02, -9.59994365e-03,  1.89045668e-02,  2.20156256e-02, 1.04296235e-02,  4.42853756e-02, -1.09862518e-02,  1.55339660e-02, 2.17024330e-02, -4.06076433e-03,  2.82253250e-02, -1.43804150e-02, 6.81060879e-03,  2.42336132e-02, -1.20356837e-02, -2.86130030e-02, 5.95987029e-02,  8.22900385e-02, -2.92814523e-03, -2.80786771e-02, -1.55530889e-02, -1.26720862e-02,  5.18352538e-02, -9.45348665e-02, -7.13151123e-04,  1.45976376e-02,  2.42314935e-02,  7.27918756e-04, 1.15067856e-02, -3.24076670e-03, -1.01865055e-02, -2.41364352e-02, 1.49887558e-02, -2.51921508e-02, -5.90336472e-02,  1.79605652e-02, 4.63739224e-02, -2.56433915e-02, -1.97405703e-02,  2.92095672e-02, -3.46023068e-02, -2.02264003e-02, -1.32118966e-02, -1.35912253e-02, 2.53015254e-02, -5.09055592e-02,  3.18482565e-03, -3.80226485e-02, 3.16254012e-02, -7.34225884e-02, -1.46069124e-01, -9.40314960e-03, 1.08394744e-02,  4.54333872e-02, -5.17369248e-02,  3.76999713e-02, 2.26711687e-02,  4.00911309e-02,  2.55041961e-02,  5.65192022e-04, -2.15513241e-02,  2.19384879e-02, -3.45059782e-02,  4.55359109e-02, 4.56896704e-03, -2.33669560e-02, -3.02887280e-02,  6.08850420e-02, 2.45300177e-02,  2.40606125e-02,  2.72339229e-02,  6.88984478e-03, -4.30227630e-03, -7.75243575e-03, -1.88227482e-02,  7.43789896e-02, -9.96600464e-03,  6.63648024e-02, -6.23515360e-02, -7.00666606e-02, -2.60777902e-02, -5.30709652e-03,  1.86568033e-03,  4.86703739e-02, -1.44296940e-02,  1.09362872e-02, -2.21310705e-02,  1.04592685e-02, 1.24127809e-02, -3.27089690e-02,  1.95319913e-02, -2.89123747e-02, -5.01579279e-03,  1.30758854e-02,  4.36534435e-02,  1.51301390e-02, 3.28035615e-02,  1.91630777e-02,  3.04533802e-02, -1.89039763e-02, -1.84636805e-02,  1.00374024e-03, -2.56961808e-02, -4.05047648e-02, 5.04714297e-03, -3.24529260e-02,  1.29275182e-02,  6.61569429e-05, 2.02226080e-02,  2.19759587e-02,  1.99162010e-02,  1.09571014e-02, 4.45986092e-02,  1.74887069e-02, -5.32580987e-02,  1.72492728e-04, -5.38621619e-02,  1.20012527e-02, -2.01341659e-02,  1.04744546e-02, 1.34008070e-02, -3.57121192e-02,  3.88850905e-02, -5.42614348e-02, -1.90397184e-02,  2.95135677e-02, -3.42997424e-02, -4.99790721e-03, 8.60128850e-02,  2.37183720e-02,  2.86867414e-02, -1.79583840e-02, -7.38559617e-03,  5.38222939e-02,  4.61249128e-02,  7.04616010e-02, 1.41099785e-02, -4.96414315e-04,  9.29754507e-03, -3.82086150e-02, 1.39192184e-02,  2.26834547e-02,  1.01052104e-02,  9.58356913e-03, -2.27210857e-02, -2.94213556e-02, -7.82643352e-03, -1.66488234e-02, -9.44784377e-03,  3.56253237e-02,  1.13743758e-02,  3.67637873e-02, -1.71556808e-02,  1.45752802e-02, -7.41183162e-02, -1.02351904e-02, 4.72791940e-02,  9.87325143e-03,  4.21396792e-02, -3.39988689e-03, -4.57472429e-02, -5.81190810e-02, -2.32971762e-03,  3.74609232e-02, -3.66091728e-02, -2.70161405e-02, -3.99984568e-02, -1.26657430e-02, 3.62169091e-03,  1.37003260e-02, -1.19137971e-04,  3.54218693e-03, -2.26079952e-02, -3.18501964e-02,  5.62375076e-02, -2.67932564e-02, -1.21969115e-02, -3.08663696e-02, -6.37506843e-02, -1.23763066e-02, -3.85162383e-02, -1.60970073e-02, -1.88369378e-02,  5.61897494e-02, 1.52650988e-02, -7.10332319e-02, -4.49051186e-02,  4.17606756e-02, 4.91539724e-02,  1.21320277e-01,  2.65271459e-02,  2.31840499e-02, 1.15248952e-02,  2.47847717e-02, -2.54846532e-02,  2.77435537e-02, -2.07310710e-02,  4.05712193e-03,  1.01060709e-02,  1.10278511e-02, -2.04517208e-02, -8.31052568e-03, -8.33136216e-03,  4.86731622e-03, 1.38870850e-02, -1.80860292e-02,  6.39171004e-02, -3.62132750e-02, -1.53473103e-02,  2.90266182e-02,  1.01663947e-01, -6.53519644e-04, -6.04799576e-02,  1.92036871e-02,  1.27083687e-02, -8.96287244e-03, -4.05058414e-02, -5.20501323e-02,  4.02576253e-02,  3.52448854e-03, -2.20351247e-03, -2.05740128e-02,  3.82163152e-02, -2.97404267e-02, -3.58801670e-02,  3.58176492e-02,  2.51023937e-02,  2.89637502e-02, 3.27022783e-02,  2.23609824e-02,  5.98784201e-02,  6.23898134e-02, 1.07959574e-02,  5.99581189e-02, -2.52771825e-02, -3.60102393e-02, 3.81944813e-02,  1.31000252e-02, -3.57606262e-02,  1.95361394e-02, 3.43371592e-02, -3.59156504e-02,  4.64263707e-02,  7.47849140e-03, -2.85285488e-02,  2.93406332e-03,  6.56284466e-02,  1.29735116e-02, 1.24560520e-02, -5.08223148e-03, -1.02384610e-03, -4.41986658e-02, 1.62261128e-02,  1.92464534e-02,  1.16519704e-01,  2.90390477e-02, 9.47278365e-03, -4.93483655e-02, -1.72430475e-03,  3.76843326e-02, 6.50802180e-02, -3.66192758e-02, -2.50096843e-02, -4.26620021e-02, -1.34685393e-02,  5.50172776e-02, -2.76150219e-02,  2.52469238e-02, 3.18296216e-02, -3.14880908e-02, -1.40442792e-03, -6.13821484e-02, 3.07383598e-03, -2.92904209e-02,  1.14536854e-02, -3.03985961e-02, -6.07095212e-02,  6.29598051e-02,  4.26955670e-02,  4.85977158e-02, 3.74925360e-02,  4.74338420e-02,  4.36811596e-02,  7.18751410e-03, 2.66182031e-02, -1.75453406e-02, -5.01529798e-02,  2.43295953e-02, 6.73195114e-03, -8.34603608e-02, -1.65526681e-02,  2.05215290e-02, -1.32691357e-02, -6.70776516e-02, -4.31945808e-02,  2.31816266e-02, 1.09050423e-03,  2.27937270e-02, -1.91466529e-02, -9.33474803e-04, -3.29071912e-03,  5.57969064e-02, -4.72763628e-02,  8.58246163e-02, -2.87788399e-02, -5.85252829e-02,  3.27921589e-03, -2.85337213e-02, -5.56366751e-03, -4.36278582e-02, -3.53846438e-02, -3.15264873e-02, -1.54559398e-02, -5.99277439e-03,  2.26277858e-02,  2.21081451e-02, 3.32079083e-02,  5.43615222e-03, -2.63609700e-02, -2.17014011e-02, -1.56563744e-02, -5.15714958e-02, -1.18470462e-02,  9.13077407e-03, -3.29162180e-02, -4.87432145e-02, -1.53248115e-02, -2.59139016e-02, -5.03621586e-02, -3.27638933e-03,  1.30307525e-02,  1.59115363e-02, 3.17716822e-02, -1.08904252e-02,  3.35477921e-03,  4.80098538e-02, 4.10186350e-02, -4.31699045e-02,  1.78821647e-04,  7.64952749e-02, -2.16004103e-02, -7.06575140e-02, -4.08859830e-03,  5.81064820e-02, -6.29717857e-03,  7.91376680e-02, -2.37569329e-03, -2.12357566e-02, -5.21706827e-02,  3.16610979e-03,  2.22804025e-02, -4.81440965e-03, 9.12775099e-03, -1.45659773e-02, -2.43284404e-02,  2.55279113e-02, -1.48536442e-02, -3.03241257e-02,  3.34586538e-02, -2.24296972e-02, 3.02538127e-02,  4.30741794e-02,  1.01597176e-03, -2.10531484e-02, 2.86896043e-02, -6.38493225e-02, -2.05329736e-03, -1.82789490e-02, 1.55664505e-02,  9.86565053e-02,  6.35040738e-03, -2.68401410e-02, 6.70046881e-02,  5.00495397e-02, -3.30492780e-02,  3.69632058e-02, 9.45759937e-03,  2.34576706e-02, -2.00130064e-02, -4.30487208e-02, 1.74981970e-02, -3.20073813e-02,  2.88598910e-02,  8.17995071e-02, 4.89620084e-04, -4.20734249e-02,  3.48180123e-02,  5.11991279e-03, -1.73648857e-02,  1.03966352e-02,  3.10861529e-03, -1.72180422e-02, 5.10167284e-03,  2.48009302e-02, -5.66994138e-02, -7.50018563e-03, 1.01825185e-02,  3.93441087e-03,  2.16627177e-02, -2.06915103e-02, -4.18055169e-02, -1.45106604e-02, -1.27682080e-02,  9.98790935e-03, -3.73138972e-02,  7.35658268e-03, -5.62764667e-02,  4.28845584e-02, -1.20785451e-02, -2.26719789e-02,  5.04938252e-02,  3.59685905e-02, -8.81960690e-02, -1.59423053e-02,  4.72089276e-02, -3.27063128e-02, 4.45395522e-03, -3.42608895e-03,  7.66208544e-02, -1.07176946e-02, -1.02114361e-02,  2.96150129e-02, -8.01335424e-02,  4.82664369e-02, 1.24619063e-02, -5.94091602e-02,  8.46937392e-03,  3.32079385e-03, 2.49857269e-02, -1.14885652e-02, -7.53031448e-02, -3.56945992e-02, 3.20922285e-02,  2.93184631e-02, -2.13059857e-02,  7.78757385e-04, 3.75101455e-02, -7.08620995e-02, -1.06713437e-02, -1.99566446e-02, -3.40806693e-03, -6.10401202e-03, -2.98068654e-02, -1.89677011e-02, 1.95910651e-02, -2.74215173e-03,  6.01873687e-03, -4.17252770e-03, -2.19440833e-02, -3.11483443e-02, -5.90865836e-02, -5.50597208e-03, 4.07601334e-02,  1.52067132e-02,  2.86911968e-02, -2.95186434e-02, -1.66269727e-02,  7.85411987e-03,  5.38409241e-02,  1.36708999e-02, -2.32281592e-02, -9.39203426e-03, -1.21814879e-02, -3.63057144e-02, 8.95278354e-04, -5.63667668e-03,  1.91887808e-33, -4.33566533e-02, -4.42274734e-02,  1.86333358e-02,  4.24921960e-02,  1.13257607e-02, -2.67407764e-03, -4.58217897e-02, -5.72701218e-03, -2.79605594e-02, 4.46375944e-02, -6.25908673e-02,  3.28818220e-03,  2.22959835e-02, 3.74244042e-02,  4.57203537e-02,  1.89937577e-02, -3.54345255e-02, 2.64822971e-02,  1.45690255e-02, -3.92485857e-02,  4.19178531e-02, 3.90173234e-02,  3.45542072e-03,  3.05347256e-02, -6.39075530e-04, -1.67947542e-02, -2.83922497e-02, -3.94321941e-02,  1.54187549e-02, 4.23817858e-02, -2.45234016e-02, -7.61900172e-02, -5.63705526e-03, -9.18691009e-02, -5.36023825e-02,  4.43371981e-02,  4.22315635e-02, -3.44523080e-02,  3.90661620e-02, -5.89124151e-02, -2.61526443e-02, -2.07635760e-02,  1.10223796e-02,  4.35565077e-02,  7.57555151e-03, -2.78332755e-02,  3.89529876e-02,  4.31878679e-02, -1.92326345e-02, 1.31701231e-02,  5.30364476e-02,  4.43998016e-02,  3.05684954e-02, -2.07589958e-02, -1.02366246e-02,  6.86659589e-02, -6.39676526e-02, -4.87711802e-02,  8.70148912e-02,  1.04625914e-02, -2.50921659e-02, 4.18020450e-02,  1.15561457e-02, -4.14195610e-03, -3.87131199e-02, 3.52347605e-02,  4.59192209e-02,  2.35098451e-02,  3.33773680e-02, -2.84436215e-02,  1.21649001e-02, -9.07375943e-03, -8.08896348e-02, 4.91869524e-02,  1.24516524e-03, -1.02520417e-02,  6.17114594e-04, -1.95689369e-02,  7.67664984e-02,  3.83759253e-02, -1.94907077e-02, 4.78590429e-02,  1.73946582e-02, -1.17715588e-02, -8.49358924e-03, 8.47095996e-02, -6.47542579e-03,  6.88125715e-02, -9.90658160e-03, -2.78850831e-03,  3.16407233e-02,  3.69632877e-02, -5.23581170e-03, -6.05059438e-04, -3.62135172e-02,  9.22156963e-03, -2.12242045e-02, -4.31616716e-02, -2.58213263e-02, -8.72304384e-03,  6.15395745e-03, -2.94016227e-02, -1.55878393e-02,  1.09848185e-02, -4.14459258e-02, -3.33895050e-02, -5.50609119e-02, -6.33776560e-02,  2.05588173e-02, 1.21245971e-02,  5.87851601e-03,  7.67237991e-02, -1.63708124e-02, 3.24134976e-02, -2.57338639e-02,  1.22733647e-03, -9.45925415e-02, 7.58718047e-03, -8.23877566e-03,  8.21105465e-02, -6.74882531e-03, -6.96882373e-03, -3.43753137e-02, -4.91802022e-03, -1.49338543e-02, 2.07468867e-02, -6.28467202e-02, -2.62052026e-02,  7.27858022e-02, -4.30631153e-02, -3.11611369e-02, -1.94044467e-02, -8.72308016e-03, -2.93835416e-03,  1.72045082e-02,  8.22418369e-03,  2.78199501e-02, -1.37404520e-02, -2.70269578e-03, -5.78028802e-03,  5.91702014e-02, 8.28758776e-02,  4.05108556e-02,  8.50384589e-03,  4.26244438e-02, -1.49471089e-02, -3.68858501e-02, -2.95383446e-02, -3.12358830e-02, -3.95183936e-02, -2.26417929e-02, -3.15803513e-02, -1.15995742e-02, 3.71009484e-03, -1.69728752e-02,  3.41749452e-02, -7.19584152e-02, 7.16953054e-02, -3.14729027e-02,  3.83929387e-02, -3.83464759e-03, -1.10111488e-02, -1.89644080e-02,  2.59531336e-03, -3.37312147e-02, 3.88063886e-03,  3.57762240e-02, -5.93993589e-02, -3.35881971e-02, -5.47134764e-02, -5.87828048e-02,  2.23364085e-02,  4.12895270e-02, 4.21102112e-03,  5.01548871e-02,  1.46141611e-02, -1.60785783e-02, 5.77300135e-03, -3.51868495e-02, -1.82182658e-02,  8.88655055e-03, 3.33121940e-02, -2.53670122e-02,  3.28676659e-03,  7.61871366e-03, -1.45879644e-03, -3.47302225e-03,  4.92487587e-02, -5.55533059e-02, 7.93799758e-02, -3.27416360e-02, -3.92518155e-02,  3.62891778e-02, -1.44601474e-02, -2.82487813e-02, -2.82876007e-02,  6.23706505e-02, -1.22910319e-02, -1.53047586e-04,  3.62271890e-02,  4.88874204e-02, -4.92698178e-02, -3.61629785e-03, -5.03963567e-02, -3.29884999e-02, -3.31856944e-02, -4.99778017e-02, -3.85035314e-02,  4.63582315e-02, 4.76273410e-02, -2.65436415e-02, -1.74311179e-04, -4.35930565e-02, -1.52840372e-02, -1.01632727e-02,  3.76403667e-02,  4.97345328e-02, 2.29855012e-02, -2.10566702e-03, -1.36193829e-02,  9.54141468e-03, -2.36655977e-02,  7.74009451e-02, -7.66901346e-03,  1.48809124e-02, 4.60187905e-02, -1.06680663e-02, -2.11192295e-02, -3.06416098e-02, -7.30406046e-02, -5.52300289e-02, -3.10481731e-02,  8.47726478e-04, 7.79758382e-04,  3.68176624e-02,  4.00339290e-02,  2.02554967e-02, -1.57325473e-02,  2.30250582e-02,  2.21156999e-02,  4.23344672e-02, 2.67792940e-02,  5.36791161e-02, -3.57918330e-02, -1.78055055e-02, -5.42198494e-03, -5.47904484e-02, -4.75045741e-02, -1.97625849e-02, 2.13571619e-02,  3.82434092e-02, -1.73607767e-02,  4.52133827e-03, 8.48614797e-03, -7.99214020e-02, -2.59474106e-02,  2.81057004e-02, -8.50231387e-03,  2.01026630e-02,  2.83441357e-02,  2.13191882e-02, 3.70609835e-02, -2.01003812e-02,  2.27825530e-02, -3.87140103e-02, -6.00847974e-02,  3.00723668e-02, -1.39068738e-02, -1.29560912e-02, 1.14637606e-01,  4.56365151e-03, -2.91780233e-02, -1.05033480e-02])
    np.testing.assert_almost_equal(student_sol_func(test_str), q2a)
    test_print('**q2a** passed sanity check!')

In [ ]:
public_test_q2a(sentence_to_sentence_embedding)

**q2a** passed sanity check!

#### Q2 (b). method 2

Like `method_one()` you implemented before. Here we do it but not with GloVe embeddings. Do what you did for Q1 (c), but use `sentence_to_sentence_embedding()`, **not** `sentence_to_average_glove()`.

The thing your function must return is a dictionary mapping each urban dictionary term to a WordNet synset ID, e.g.:

```
{
 "adore" : "love.v.01",
 "dripping" : "stylish.a.01"    
 }
```

New to Q2: here, you *must* add appe d similarity scores to the list `all_term_similarity_scores` provided, and as shown in the template code, return that variable as well.

In [ ]:
def method_two(sentence_encoder, urban_dictionary_terms: List[Tuple[str, str]], target_synsets: List[str]):
    """
    Method 2: an algorithm based on sentence embeddings that maps each urban dictionary term to a synset ID.

    Parameters
    ----------
    sentence_encoder :
        a function to encode a sentence into word embeddings
        You should pass the function `sentence_to_sentence_embedding` implemented in q2a here
    urban_dictionary_terms : List[Tuple[str, str]]
        a list of string 2-tuples where the first elements are words, second elements are definitions.
    target_synsets : List[str]
        a list of synset IDs that the words should be classified into.
        You can call `wn.synset("<synset ID>")` to get the synset object.

    Returns
    --------
    A dictionary mapping each urban dictionary term to a WordNet synet ID, e.g.
    `{"adore" : "love.v.01", "dripping" : "stylish.a.01"}`

    """
    all_term_similarity_scores=[]
    # BEGIN SOLUTION
    best_matches = {}
    for word, definition in urban_dictionary_terms:
        cosine_sword = get_term_similarity_scores(sentence_encoder, word, definition, target_synsets, silent=True)
        all_term_similarity_scores.append(cosine_sword)
        best_matches[word] = cosine_sword[0][0] # TODO
    # END SOLUTION
    return best_matches, all_term_similarity_scores

In [ ]:
method_two_results, all_term_similarity_scores=method_two(sentence_to_sentence_embedding, urban_dictionary_terms, target_synsets)
method_two_results

{'Crowdfunding': 'fund-raise.v.01',
 'Hygge': 'coziness.n.01',
 'biohacking': 'mutation.n.02',
 'TL;DR': 'kernel.n.03',
 'Hellacious': 'formidable.s.01',
 'Unfriend': 'stranger.n.02',
 'Infodemic': 'spread.n.01',
 'Onboarding': 'orientation.n.06',
 'Truthiness': 'plausibility.n.01',
 'Amotivational': 'faineant.s.01',
 'NSFW': 'faineant.s.01',
 'Rando': 'stranger.n.02'}

In [ ]:
def public_test_q2b(student_sim_scores):
    np.testing.assert_almost_equal(student_sim_scores[-1][-5][-1], 0.12022592)
    test_print('**q2b** passed sanity check!')

In [ ]:
public_test_q2b(all_term_similarity_scores)

**q2b** passed sanity check!

### Deliverable 3 (TODO): Define an evaluation metric (accuracy).  

Throughout this semester we've stressed how critical evaluation is for any NLP method.  Implement a function `accuracy` that assesses quality of the dictionaries you return from `method_one` and `method_two`.  This accuracy function should return a single real number (the accuracy), and its input parameters are a prediction dict (the output of your model) and a truth dict (which you will need to create based on your own judgement). Make sure that the accuracies you calculate for the two methods match what you expect.


In [ ]:
def accuracy(prediction: Dict[str, str], truth: Dict[str, str]) -> float:
    # BEGIN SOLUTION
    accurate_pred = 0
    for word, definition in prediction.items():
      if word in truth.keys():
        if truth[word] == definition:
            accurate_pred +=1
    return accurate_pred/len(prediction) # TODO
    # END SOLUTION


In [ ]:
truth= {'Crowdfunding': 'spread.n.01',
 'Hygge': 'stranger.n.02',
 'biohacking': 'kernel.n.03',
 'TL;DR': 'stranger.n.02',
 'Hellacious': 'formidable.s.01',
 'Unfriend': 'stranger.n.02',
 'Infodemic': 'spread.n.01',
 'Onboarding': 'orientation.n.06',
 'Truthiness': 'stranger.n.02',
 'Amotivational': 'kernel.n.03',
 'NSFW': 'stranger.n.02',
 'Rando': 'stranger.n.02'}

In [ ]:
print(accuracy(method_one_results, truth))

1.0


In [ ]:
print(accuracy(method_two_results, truth))

0.4166666666666667


In [ ]:
def public_test_q3(student_sol_func):
    test={
        'NSFW': 'inappropriate.a.01',
        'biohacking': 'mutation.n.02'
    }
    np.testing.assert_array_less(student_sol_func(method_one_results, test),
                                 student_sol_func(method_two_results, test))
    test_print('**q3** passed sanity check!')

In [ ]:
public_test_q3(accuracy)

**q3** passed sanity check!

---

**Congrats!** That's it for HW5.

Submit your work to “HW5 (ipynb)” on Gradescope. Here are some recommended steps:

- Restart your Colab runtime and re-run all your cells; make sure everything runs as expected.

- Remember all your work MUST BE PLACED between `# BEGIN SOLUTION` and `# END SOLUTION`.

- Download your Colab notebook as an `.ipynb file` (File → Download .ipynb)

- Upload HW5.ipynb to Gradescope.


### Optional

Use the two methods you've defined to find the best-matching synset within the **entire** WordNet. Do the results make sense? Is one method consistently better than the other?

Here's a reminder of how to iterate through all synsets:

In [ ]:
for idx, synset in enumerate(wn.all_synsets()):
    print(idx, synset)
    if (idx > 10): break

0 Synset('able.a.01')
1 Synset('unable.a.01')
2 Synset('abaxial.a.01')
3 Synset('adaxial.a.01')
4 Synset('acroscopic.a.01')
5 Synset('basiscopic.a.01')
6 Synset('abducent.a.01')
7 Synset('adducent.a.01')
8 Synset('nascent.a.01')
9 Synset('emergent.s.02')
10 Synset('dissilient.s.01')
11 Synset('parturient.s.02')
